# Using inaFaceAnalyzer API: a quick-start tutorial
In this tutorial, we use inaFaceAnalyzer with default analysis parameters and export results to CSV, rich ASS subtitles and incrusted MP4.

## Install inaFaceAnalyzer

In [1]:
# try to import inaFaceAnalyzer and import it from Pypi's
# if it is not available
try:
  import inaFaceAnalyzer
except:
  # install inaFaceAnalyzer Pypi's distribution
  !pip install inaFaceAnalyzer

## Define and display sample video input

In [2]:
from IPython.display import Video

# input materials can be provided using file paths or remote urls
sample_vid = 'https://github.com/ina-foss/inaFaceAnalyzer/raw/master/media/pexels-artem-podrez-5725953.mp4'
# set desired width used for displaying video
nbw = 600
# Display remote input video
Video(sample_vid, width=nbw)

## Analyse a video with default parameters

In [3]:
#import the video processing engine
from inaFaceAnalyzer.inaFaceAnalyzer import VideoAnalyzer

In [4]:
# create a video analyzer instance
# classifications models may be downloaded from remote location
# machine learning models are loaded during analyzer constructor and may require several seconds
# consequently, users should use the same analyzer instance to process several documents
va = VideoAnalyzer()

2022-02-19 17:12:20.042711: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-19 17:12:20.043957: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-19 17:12:20.044197: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-19 17:12:20.044509: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
# Analyzers are designed as 'functions objects' or 'functors'
# see https://en.wikipedia.org/wiki/Function_object
# ie: analyzer instances can be used as functions, doing the code implemented in __call__ methods
# this example will have "long" processing time since all video frames will be processed
df = va(sample_vid)

2022-02-19 17:12:26.741315: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8300
2022-02-19 17:12:26.745917: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-02-19 17:12:27.841156: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-02-19 17:12:27.841210: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-02-19 17:12:27.896017: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.4

In [6]:
# Analysis results are returned as pandas DataFrames
# see https://pandas.pydata.org/docs/
# Results one line per detected faces and several columns : 
#
# frame: the position of the frame in the video stream
# bbox: (left, top, right, bottom) the bounding box of the face in the image frame
# detect_conf: the face detection confidence estimate (dependent on the face detection method used)
# sex_decfunc: raw gender classifier output : positive values are used for men and negative values for women
# sex_label: gender classifer prediction: 'm' for men and 'w' for 'women'
# age_decfunc: raw age regression output based on FairFace age categories.
# 0 for (0-3 years old), 1 for (4-9) years, 2 for (10-19)  years, 3 for (20-29)  years, etc...
# sex_label : "human-readable" age prediction
df

,frame,bbox,detect_conf,sex_decfunc,age_decfunc,sex_label,age_label
0,0,"(945, -17, 1139, 177)",0.999999,8.408013,3.912696,m,34.126964
1,0,"(71, 119, 272, 320)",0.999996,-13.514767,3.124181,f,26.241808
2,0,"(311, 163, 491, 343)",0.999979,-11.023161,3.035822,f,25.358219
3,0,"(558, 202, 728, 371)",0.999974,8.824918,2.910231,m,24.102311
4,0,"(745, 23, 930, 208)",0.981539,-7.944937,2.427218,f,19.272180
...,...,...,...,...,...,...,...
1309,356,"(612, 262, 795, 445)",0.999994,12.959448,2.908627,m,24.086266
1310,356,"(343, 120, 479, 256)",0.999957,-8.931982,2.968017,f,24.680171
1311,357,"(876, 65, 1013, 203)",0.999999,-7.740976,2.653409,f,21.534090
1312,357,"(610, 260, 797, 447)",0.999997,12.895056,2.973207,m,24.732065


## Exporting analysis results

In [7]:
# dataframes are easy to export in CSV or to any tabular format
df.to_csv('./myexport.csv')
# display the resulting csv
!cat myexport.csv

,frame,bbox,detect_conf,sex_decfunc,age_decfunc,sex_label,age_label
0,0,"(945, -17, 1139, 177)",0.999998927116394,8.408013,3.9126964,m,34.12696361541748
1,0,"(71, 119, 272, 320)",0.9999958872795105,-13.514767,3.1241808,f,26.24180793762207
2,0,"(311, 163, 491, 343)",0.99997878074646,-11.023161,3.035822,f,25.358219146728516
3,0,"(558, 202, 728, 371)",0.9999741911888123,8.824918,2.910231,m,24.10231113433838
4,0,"(745, 23, 930, 208)",0.9815391302108765,-7.9449368,2.427218,f,19.27217960357666
5,1,"(946, -17, 1138, 175)",0.9999986290931702,7.916045,3.872796,m,33.72796058654785
6,1,"(66, 117, 274, 324)",0.9999975562095642,-12.532553,3.1050837,f,26.05083703994751
7,1,"(558, 202, 730, 373)",0.9999759793281555,7.8327827,2.8403559,m,23.4035587310791
8,1,"(311, 164, 491, 344)",0.9999755620956421,-10.973633,2.989624,f,24.896240234375
9,2,"(947, -15, 1139, 177)",0.9999986290931702,9.031829,3.7397985,m,32.39798545837402
10,2,"(68, 118, 273, 323)",0.9999967217445374,-12.308103,3.288724,f,27.8872394561

## Exporting results to rich subtitles
Ass subtitles allow to display detected faces and classification results in VLC or ELAN
Subtitles are a good option for sharing results, since they do not require a large amount of storage size, and do not alter original videos

In [8]:
from inaFaceAnalyzer.display_utils import ass_subtitle_export

In [9]:
# export results to ass
# requires
# arg1 : input video filename or URL
# arg2 : analysis result (as a pandas dataframe OR as a resulting csvfile)
# arg3 : export ass subtitle file name
# arg4: analysis FPS not used here
ass_subtitle_export(sample_vid, df, './mysample.ass')

In [10]:
# download video: VLC cannot display subtitles on a remote video
!wget $sample_vid -O ./sample_vid.mp4
# open original video with the subtitle file in VLC (cannot be done in google collab)
!vlc --sub-file ./mysample.ass ./sample_vid.mp4

--2022-02-19 17:12:46--  https://github.com/ina-foss/inaFaceAnalyzer/raw/master/media/pexels-artem-podrez-5725953.mp4
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ina-foss/inaFaceAnalyzer/master/media/pexels-artem-podrez-5725953.mp4 [following]
--2022-02-19 17:12:46--  https://raw.githubusercontent.com/ina-foss/inaFaceAnalyzer/master/media/pexels-artem-podrez-5725953.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4198252 (4,0M) [application/octet-stream]
Saving to: ‘./sample_vid.mp4’

./sample_vid.mp4    100%[===================>]   4,00M  --.-KB/s    in 0,09s   

2022-02

[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[ass] fontselect: (DejaVu Sans, 400, 0) -> /usr/share/fonts/truetype/dejavu/DejaVuSans.ttf, 0, DejaVuSans
[ass] fontselect: (Arial, 400, 0) -> /usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf, 0, LiberationSans
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420]

[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture he

[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture heap full
[00007fdfac0a1420] main subpicture error: subpicture he

## Exporting results to incrusted MP4 Videos
We provide result export options to MP4
MP4 export is slow and generate large files, we recommed using ASS subtitle export when possible

In [11]:
from inaFaceAnalyzer.display_utils import video_export

In [12]:
# export results to MP4
# requires
# arg1 : input video filename or URL
# arg2 : analysis result (as a pandas dataframe OR as a resulting csvfile)
# arg3 : resulting incrusted MP4 file name
# arg4: analysis FPS not used here
video_export(sample_vid, df, './myexportedvid.mp4')

ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-l

In [13]:
Video("./myexportedvid.mp4", width=nbw)